## Config

In [16]:
### Configuration
# Dependencies
import os
import pandas as pd
import numpy as np
from summerplaylist import *

# Environment variables
CLIENTID = os.environ.get('CLIENTID')
CLIENTSECRET = os.environ.get('CLIENTSECRET')
REDIRECT_URI = 'https://example.com/callback'

# Filepaths
artistsKnownPath = os.path.join('data', 'artistsKnown.csv')

# Read in known artists
artistsKnown = pd.read_csv(artistsKnownPath)

# Set max age for new artist songs
maxAge = 5

## Log In

In [2]:
# Authenticate to Spotify
credentials = authenticateToSpotify(CLIENTID, CLIENTSECRET, REDIRECT_URI)

Opening browser for Spotify login...


## Collect new artists

In [9]:
# Collect names of new artists from Music Festival Wizard
artistsNew = fetchMusicFestivalWizard()
# Filter out known artists who we have already listened to
artistsNew = artistsNew.loc[~artistsNew['artistName'].isin(artistsKnown['artistName'])]
artistsNew = artistsNew.head()
# Search spotify for artistIDs
artistsNew['artistID'] = artistsNew.apply(findArtistIDs, credentials=credentials, axis=1)
# Filter out if artistID was known
artistsNew = artistsNew.loc[~artistsNew['artistID'].isin(artistsKnown['artistID'])]
artistsNew

,artistName,artistID


In [14]:
# Collect artists from an existing favorited playlist
playlistToFetch = getPlaylistID(credentials, 'anti pop')
artistsNew = fetchTracksFromPlaylist(credentials, playlistToFetch)
artistsNew = artistsNew[['artistName', 'artistID']].drop_duplicates('artistID', ignore_index=True)
# Filter out if artistID was known
artistsNew = artistsNew.loc[~artistsNew['artistID'].isin(artistsKnown['artistID'])]

## Process new artists

In [15]:
artistsNew


,artistName,artistID
0,sagacious,0wzoSAAL4Yq5ph8q28HXEU
1,Tommy Newport,7FvQR6QFfXmW18dyMqnHvv
2,Mercury,6aIZIwesnYKR6ZN1fIerPZ
3,Portraits Of Tracy,34QizBjxVeMcA5HlZpKxxE
5,annonXL,7G3FmxYGrHlmOMqoG2lHdA
...,...,...
109,Dev Lemons,5zbFRbEu3FDkqN4j1awLLO
110,bodyimage,7pn8MYeH0VgKGDnYAiS7Pg
111,Sophia Black,3wdXgSpjbbU3tX9KasdxMh
112,King Isis,7jz1wM1NdiskFWrlosZhVw


## Get top songs

In [17]:
# Get their top three songs in last 5 years
songURIs = fetchTopSongs(credentials, artistsNew['artistID'], maxAge=maxAge)
# How many songs are in this round?
print(f"There are {len(songURIs)} songs about to be added")

There are 266 songs about to be added


## Put songs in playlist

In [18]:
# Warning: This will make a new playlist
playlistID = getPlaylistID(credentials, 'New Artists to Explore')
# Add songs to the playlist
addToPlaylist(credentials, songURIs, playlistID)

## Update the list of known artists

In [19]:
# Append artistsNew to artistsKnown.csv 
allArtists = pd.concat([artistsKnown, artistsNew], axis=0, ignore_index=True)
allArtists.to_csv(artistsKnownPath, index=False)

## Create playlist to drop good songs into

In [21]:
# Cull exploreArtists into ArtistHasPotential playlist
winnersPlaylist = getPlaylistID(credentials, 'New Artists to Explore Further')
print(f'Now drop songs you like into this playlist: {winnersPlaylist}')

Now drop songs you like into this playlist: 0Yj1Gqn15HdcDNcV5XJqVm
